## Hash set membership demo

In [27]:
# check if notebook is in colab
try:
    # install ezkl
    import google.colab
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "ezkl"])
    subprocess.check_call([sys.executable, "-m", "pip", "install", "onnx"])

# rely on local installation of ezkl if the notebook is not in colab
except:
    pass


# here we create and (potentially train a model)

# make sure you have the dependencies required here already installed
from torch import nn
import ezkl
import os
import json
import torch


class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()

        # this is a constant set
        self.set = torch.nn.Parameter(torch.tensor([0.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0]))

    def forward(self, x):
        diff = (x - self.set)
        membership_test = torch.prod(diff, dim=1)
        return membership_test


circuit = MyModel()

# Train the model as you like here (skipped for brevity)




In [28]:
model_path = os.path.join('network.onnx')
compiled_model_path = os.path.join('network.compiled')
pk_path = os.path.join('test.pk')
vk_path = os.path.join('test.vk')
settings_path = os.path.join('settings.json')
srs_path = os.path.join('kzg.srs')
witness_path = os.path.join('witness.json')
data_path = os.path.join('input.json')

In [29]:
# print pytorch version 
print(torch.__version__)

2.0.1


In [30]:


# After training, export to onnx (network.onnx) and create a data file (input.json)
# hash(0) = 0x00000000, so this will be a member of the set
x = 0.1*torch.zeros(1,*[1], requires_grad=True)

# Flips the neural net into inference mode
circuit.eval()

    # Export the model
torch.onnx.export(circuit,               # model being run
                      x,                   # model input (or a tuple for multiple inputs)
                      model_path,            # where to save the model (can be a file or file-like object)
                      export_params=True,        # store the trained parameter weights inside the model file
                      opset_version=14,          # the ONNX version to export the model to
                      do_constant_folding=True,  # whether to execute constant folding for optimization
                      input_names = ['input'],   # the model's input names
                      output_names = ['output'], # the model's output names
                      dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                    'output' : {0 : 'batch_size'}})

data_array = ((x).detach().numpy()).reshape([-1]).tolist()

data = dict(input_data = [data_array])

    # Serialize data into file:
json.dump( data, open(data_path, 'w' ))


================ Diagnostic Run torch.onnx.export version 2.0.1 ================
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



In [35]:
run_args = ezkl.PyRunArgs()
# "hashed/private" means that the output of the hashing is not visible to the verifier and is instead fed into the computational graph
run_args.input_visibility = "hashed/private"
# we set it to fix the set we want to check membership for
run_args.param_visibility = "public"
# the output is public -- set membership fails if it is not = 0
run_args.output_visibility = "public"
run_args.variables = [("batch_size", 1)]
# never rebase the scale
run_args.scale_rebase_multiplier = 1000
# logrows
run_args.logrows = 11

#  this creates the following sequence of ops:
# 1. hash the input -> poseidon(x)
# 2. compute the set difference -> poseidon(x) - set
# 3. compute the product of the set difference -> prod(poseidon(x) - set)


# TODO: Dictionary outputs
res = ezkl.gen_settings(model_path, settings_path)
assert res == True


In [36]:
res = ezkl.compile_model(model_path, compiled_model_path, settings_path)
assert res == True

In [37]:
# srs path
res = ezkl.get_srs(srs_path, settings_path)

In [38]:
# now generate the witness file 

res = ezkl.gen_witness(data_path, compiled_model_path, witness_path, settings_path = settings_path)
assert os.path.isfile(witness_path)

In [39]:

# HERE WE SETUP THE CIRCUIT PARAMS
# WE GOT KEYS
# WE GOT CIRCUIT PARAMETERS
# EVERYTHING ANYONE HAS EVER NEEDED FOR ZK



res = ezkl.setup(
        compiled_model_path,
        vk_path,
        pk_path,
        srs_path,
        settings_path,
    )

assert res == True
assert os.path.isfile(vk_path)
assert os.path.isfile(pk_path)
assert os.path.isfile(settings_path)

inputs: [Value { inner: Tensor { inner: [Value(Value { inner: Some(0x0000000000000000000000000000000000000000000000000000000000000000) })], dims: [1], scale: None, visibility: None }, dims: [1], scale: 1 }]
inputs: [Value { inner: Tensor { inner: [Value(Value { inner: Some(0x0000000000000000000000000000000000000000000000000000000000000000) })], dims: [1], scale: None, visibility: None }, dims: [1], scale: 1 }]


In [40]:
# GENERATE A PROOF


proof_path = os.path.join('test.pf')

res = ezkl.prove(
        witness_path,
        compiled_model_path,
        pk_path,
        proof_path,
        srs_path,
        "evm",
        "single",
        settings_path,
    )

print(res)
assert os.path.isfile(proof_path)

inputs: [Value { inner: Tensor { inner: [Value(Value { inner: Some(0x0000000000000000000000000000000000000000000000000000000000000000) })], dims: [1], scale: None, visibility: None }, dims: [1], scale: 1 }]
{'instances': [[[0, 0, 0, 0]], [[0, 0, 0, 0]]], 'proof': '050a135a2ec931e6d68e3ae401492a79f17942769f9c9786ff8b465b3fb7e0900e778a49086f48687e44962846de405dd1979a8fe45eb147f60c9ceb8950b51c0b9250bd9df2f54ad09b73725ca70568e8244fde8b88bf9ec27446a89a0bf9572c6ae53fb50c153acf7196d4e6d6b62a789c798ac97e34776bcaa5753bc52fa42d1f0e9004ac580c8db77aa2fa31259ab100db629c70013891c292ecc863713725fca1b69c332a2e66a3868bfa1a68e7484f92f747028c787a8ed4b8e17cb33f0f71a72e8b54a91c9e859abe8bcb6684051ac215bea6ec54d6be29badde47ef026af9d35b4298051d31bd955a4f0f958c93a245ec80c3ae3131d1bcd97a7cd370bcd86d7d7e95f358626981bebe15ee31e1c7bfc4ffa7474439bbe8377ee0813024333bddcefa1025626a8c113db0f3e932fd9719be557257db490e4fb1f21af19cc8ca38253c967ab1072a9c32589830252a9451ef0c547f63cebbad3717f2423731414f4821f58509492955913ac2

In [41]:
# VERIFY IT

res = ezkl.verify(
        proof_path,
        settings_path,
        vk_path,
        srs_path,
    )

assert res == True
print("verified")

verified
